In [3]:
import csv
import requests
from bs4 import BeautifulSoup

base_url = "http://books.toscrape.com/catalogue/page-{}.html"
page_number = 1
with open("books_data.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["title", "price", "availability", "rating"])
    while True:
        url = base_url.format(page_number)
        response = requests.get(url)
        if response.status_code != 200:
            break  
        soup = BeautifulSoup(response.content, "html.parser")
        books = soup.find_all("article", class_="product_pod")
        for book in books:
            title = book.h3.a["title"]
            price = book.find("p", class_="price_color").text[1:]
            availability = book.find("p", class_="instock availability").text.strip()
            rating_class = book.find("p")["class"][1]
            rating_dict = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
            rating = rating_dict.get(rating_class, 0)
            writer.writerow([title, price, availability, rating])
        page_number += 1  

print("Data scraping completed! Check 'books_data.csv' for results.")

Data scraping completed! Check 'books_data.csv' for results.


In [1]:
#VERY IMPORTANT STEP- install this, used to connect mysql workbench with Jupyter
!pip install mysql-connector-python 



In [7]:
import pandas as pd
import mysql.connector

df = pd.read_csv("books_data.csv")
df["price"] = df["price"].astype(float)
df["availability"] = df["availability"].apply(lambda x: 1 if "In stock" in x else 0)
conn = mysql.connector.connect(
    host="localhost",      
    user="root",          
    password="##", # Your MySQL password
    database="books_db"
)
cursor = conn.cursor()
for _, row in df.iterrows():
    cursor.execute("INSERT INTO books (title, price, availability, rating) VALUES (%s, %s, %s, %s)",
                   (row["title"], row["price"], row["availability"], row["rating"]))

conn.commit()
cursor.close()
conn.close()

print("✅ Data successfully inserted into MySQL!")


✅ Data successfully inserted into MySQL!
